In [ ]:
with open (“./passwords.json”, “r”)
as f:
sensitive_data = json.load(f)


In [16]:
from elasticsearch import helpers 
from elasticsearch.helpers import scan
from collections.abc import MutableMapping
import pandas as pd
import numpy as np

def flatten(nested_dict, parent_key=''):
    items = []
    for k, v in nested_dict.items():
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, k).items())
        else:
            items.append((k, v))
    return dict(items)

query = {
  "sort": [
    {
      "timestamp": "desc"
    }
  ],
  "query": {
    "match_phrase": {
      "event": "Search result selected"
    }
  }
 }


#note: scan works fast because it grabs data unsorted.
#grabs 100,000 without scan. seems to have trouble past 500,000
response = helpers.scan(es,
    query=query, preserve_order=True, 
    index="search_relevance_implicit",
)
n_events_to_fetch = 250000


df = pd.DataFrame([
    flatten(next(response)['_source']) 
    for _ in range(n_events_to_fetch)
])


In [17]:
#note: Python client automatically indexes from latest to earliest.  

#"timestamp": datetime(2010, 10, 10, 10, 10, 10)

#To check start date:

sorted=df.sort_values(by=['timestamp'], ascending=True) 
sorted.head()

,event,anonymousId,timestamp,network,toggles,aggregations,items.locations.locationType,items.locations.type,page,production.dates.from,...,position,resultIdentifiers,resultLanguage,resultSubjects,resultWorkType,source,_queryType,color,items.locations.accessConditions.status,genres.label
199999,Search result selected,acda18fb-b1e0-4419-ae57-78fab56fc022,2020-06-17T05:49:00.747Z,None,[],[workType],"[iiif-image, iiif-presentation]",NaN,1,None,...,15,"[b14659621, 1465962, V0025386]",English,"[Optics., Lenses.]",Pictures,work_result,BoolBoosted,NaN,NaN,NaN
199998,Search result selected,acda18fb-b1e0-4419-ae57-78fab56fc022,2020-06-17T05:53:02.425Z,None,[],[workType],"[iiif-image, iiif-presentation]",NaN,2,None,...,3,[B0008487],NaN,[],Digital Images,work_result,BoolBoosted,NaN,NaN,NaN
199997,Search result selected,99268b47-0e41-4090-b703-6e9817470d3d,2020-06-17T05:53:20.727Z,None,[],[workType],"[iiif-image, iiif-presentation]",NaN,1,None,...,3,"[b12040502, 1204050, V0024748]",English,"[Astronomy., Trees., Moon.]",Pictures,work_result,PhraserBeam,NaN,NaN,NaN
199996,Search result selected,206c77da-b74f-4994-bf7d-e919654a4837,2020-06-17T05:55:09.206Z,None,[],[workType],"[iiif-image, iiif-presentation]",NaN,1,None,...,10,"[L0040553, b16554097, EPH 557:5]",NaN,"[Advertisement, Product]",Digital Images,work_result,BoolBoosted,NaN,NaN,NaN
199995,Search result selected,c5cca18e-58ed-4233-8c0c-d06f7f21710e,2020-06-17T05:56:33.866Z,None,[],[workType],"[iiif-image, iiif-presentation]",NaN,4,None,...,1,"[b11744066, 1174406, V0015904]",French,"[Blind., Street musicians., Dogs., Hats., Drum...",Pictures,work_result,BoolBoosted,NaN,NaN,NaN


In [26]:
#remove staff usage, limit time frame to 1/7/20 - 30/9/20
df2=df.loc[(df['network'] != 'StaffCorporateDevices') & (df['timestamp'] >= '2020-07-01') & (df['timestamp'] < '2020-10-01')] 

# grab only the columns needed
df2=df2[['id', 'resultWorkType', 'anonymousId', 'timestamp']]

#sort the dataframe
df2.sort_values(by=['anonymousId','id'], inplace=True)
df2.head(5)


,id,resultWorkType,anonymousId,timestamp
159678,cyzuf9pb,Digital Images,00006b57-6b25-44cd-a8a2-2ca62671fcc5,2020-07-13T21:15:48.184Z
114836,smxpuwj3,Pictures,000223d7-c1db-4852-aca1-860be7b66899,2020-08-09T22:08:45.139Z
96399,s8ckh9eu,Pictures,00038646-43b1-49d9-bb75-547e655c7ee7,2020-08-23T17:09:33.722Z
102836,nv9pfazp,Pictures,0003e566-dae1-44b7-a22c-cf5a27043e34,2020-08-18T17:41:38.969Z
102837,uudv6zd7,Pictures,0003e566-dae1-44b7-a22c-cf5a27043e34,2020-08-18T17:40:44.327Z


In [27]:
#dedupe 

df2.sort_values(by=['id','anonymousId'])
df3=df2.drop_duplicates(subset=['anonymousId','id'], keep='first')


In [28]:
#How many workIds?
summary=df3.groupby('id').count()[['anonymousId']]
print(summary)


          anonymousId
id                   
a2239muq            2
a227dajt           15
a227y9ye            3
a22au6yn           95
a22g3y27            5
...               ...
zzyxusav            1
zzzaaraw            2
zzzsptk9            2
zzzx2g6s            1
zzzyukhp            5

[56338 rows x 1 columns]


In [29]:
#How many workIds?
summary['anonymousId'].count()

56338

In [31]:
#create index for dataframe
sorted=summary.sort_values(by=['anonymousId'], ascending=False)
sorted.head()

,anonymousId
id,
mbk4nzgb,727
a2qbugvp,650
xreb5xue,414
e5qgc2dj,335
bs83y268,301


In [32]:
#output data to csv to check

summary.to_csv('selects_by_workId.csv') 